# 2a. Satellite Featurizations Part 1 - Obtaining Tile Geometries

In [12]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time
warnings.filterwarnings('ignore')

In [13]:
TILES_FOLDER = '/data/mosaiks/replication/sampled_tiles/'
SHAPEFILES_FOLDER = '/data/mosaiks/shapefiles/'
MAX_TILES_PER_REGION = 100

In [14]:
# RUN THIS CELL FOR US
SHAPEFILE_FNAME = SHAPEFILES_FOLDER + 'us_pumas/pumas.shp'
SHAPEFILE_IDS = ['State', 'PUMA']
DATA_OUTFOLDER = TILES_FOLDER + 'us/'
PLOTS_OUTFOLDER = None

In [ ]:
# RUN THIS CELL FOR MEXICO
SHAPEFILE_FNAME = '/data/mosaiks/shapefiles/mexico_municipalities.geojson'
SHAPEFILE_IDS = ['municipality']
DATA_OUTFOLDER = TILES_FOLDER + 'mexico/'
PLOTS_OUTFOLDER = None

In [ ]:
# RUN THIS CELL FOR INDIA
# Esther TODO

In [ ]:
# RUN THIS CELL FOR DHS COUNTRIES
dhs_country = 'peru' # change 'colombia' to each DHS country
SHAPEFILE_FNAME = '/data/mosaiks/surveys/dhs/' + dhs_country + '_polygons.geojson'
SHAPEFILE_IDS = ['cluster']
DATA_OUTFOLDER = TILES_FOLDER + 'dhs/' + dhs_country + '/' 
PLOTS_OUTFOLDER = None

In [15]:
# Read in shapefile and format
shapefile = gpd.read_file(SHAPEFILE_FNAME)
shapefile = shapefile.to_crs(epsg=4326)
shapefile['bounds'] = shapefile['geometry'].apply(lambda x: x.bounds)

In [16]:
# Get sampled MOSAIKS tiles for each region in the shapefile, along with degree of overlap
num_eligible_squares = []
num_used_squares = []
tiles = []
plot = False
start = time.time()
for i in range(len(shapefile)):
    
    print(('%.2f' % (100*i/len(shapefile))) + '% gridded...', end='\r')
    
    # Get the region
    region = shapefile.iloc[i]
    region_info = gpd.GeoDataFrame(pd.DataFrame(region).T)
    
    # Get the bounding box for the region
    min_latitude, max_latitude = region['bounds'][1], region['bounds'][3]
    min_longitude, max_longitude = region['bounds'][0], region['bounds'][2]
    
    # Define the latitude/longitude grid of centroids
    latitude_grid = (np.arange(round(min_latitude, 2)-.02, round(max_latitude, 2)+.02, .01) - .005)
    longitude_grid = (np.arange(round(min_longitude, 2)-.02, round(max_longitude, 2)+.02, .01) - .005)
    grid = np.meshgrid(latitude_grid, longitude_grid)
    grid = np.array([grid[0].flatten(), grid[1].flatten()])
    grid = pd.DataFrame(grid).T
    grid.columns = ['Latitude', 'Longitude']
    grid = gpd.GeoDataFrame(grid, geometry=gpd.points_from_xy(grid['Longitude'], grid['Latitude']), crs=4326)
    
    # Turn centroids into squares
    squares = grid.copy()
    squares['geometry'] = squares['geometry'].buffer(.005, cap_style=3)
    
    # Determine squares that have some overlap with the region
    eligible_squares = gpd.sjoin(squares, region_info, how='inner', op='intersects')
    num_eligible_squares.append(len(eligible_squares))
    
    # If more squares overlap with PUMA than we can process, take UAR sample of overlapping squares
    if len(eligible_squares) > MAX_TILES_PER_REGION:
        sampled_tiles = eligible_squares.sample(n=MAX_TILES_PER_REGION, replace=False, random_state=1)
    else:
        sampled_tiles = eligible_squares.copy()
    
    # Get weight for each tile (degree of overlap)
    sampled_tiles['weight'] = sampled_tiles['geometry']\
        .apply(lambda x: 100*region['geometry'].buffer(0).intersection(x).area/x.area)
    sampled_tiles = sampled_tiles.dropna(how='any')
    
    num_used_squares.append(len(sampled_tiles))
    
    # Write to file
    sampled_tiles[['Latitude', 'Longitude', 'weight'] + SHAPEFILE_IDS]\
        .to_csv(DATA_OUTFOLDER + '/' + '_'.join([str(region[key]) for key in SHAPEFILE_IDS]), index=False)
    
    if PLOTS_OUTFOLDER is not None:
        fig, ax = plt.subplots(1, figsize=(20, 20))
        region_info.plot(ax=ax)
        grid.plot(ax=ax, color='black', markersize=4)
        squares.plot(ax=ax, color='lightgrey', alpha=.1, edgecolor='black')
        eligible_squares.plot(ax=ax, color='grey', alpha=.3, edgecolor=None)
        sampled_tiles.plot(ax=ax, color='grey', alpha=.5, edgecolor=None)
        ax.axis('off')
        ax.set_title(' '.join([str(region[key]) for key in SHAPEFILE_IDS]), fontsize='xx-large')
        plt.savefig(PLOTS_OUTFOLDER + '/' + '_'.join([str(region[key]) for key in SHAPEFILE_IDS]), dpi=300)

print('Done gridding!')

Done gridding!...


In [17]:
# Get all tiles together, write to files partitioned for MOSAIKS API
tiles = []
for fname in os.listdir(DATA_OUTFOLDER):
    if fname[0] != '.':
        tiles.append(pd.read_csv(DATA_OUTFOLDER + '/' + fname))
tiles = pd.concat(tiles)
tiles = tiles[['Latitude', 'Longitude', 'weight'] + SHAPEFILE_IDS]
tiles.to_csv(DATA_OUTFOLDER + '/sampled_tiles.csv', index=False, float_format='%.3f')
tiles_with_duplicates = pd.read_csv(DATA_OUTFOLDER + '/sampled_tiles.csv')
tiles = tiles_with_duplicates.drop_duplicates(subset=['Latitude', 'Longitude'])
partitions = list(np.arange(0, len(tiles), 100000)) + [len(tiles)]
for i in range(len(partitions)-1):
    tiles[partitions[i]:partitions[i+1]]\
        .to_csv(DATA_OUTFOLDER + 'sampled_tiles_partition_' + str(i) + '.csv', index=False)

In [18]:
# Print summary statistics on tiles 
meta_info = shapefile[SHAPEFILE_IDS]
meta_info['num_eligible_squares'] = num_eligible_squares
meta_info['count'] = num_used_squares
tiles['full_overlap'] = (tiles['weight'] == 100).astype('int')
overlap = tiles.groupby(SHAPEFILE_IDS, as_index=False).agg('mean')[SHAPEFILE_IDS + ['full_overlap']]
meta_info = meta_info.merge(overlap, on=SHAPEFILE_IDS)
meta_info.to_csv(DATA_OUTFOLDER + 'meta.csv', index=False)

min_eligible_squares = meta_info.sort_values('num_eligible_squares', ascending=True).iloc[0]
print(('Minimum eligible tiles: %.2f (' +  ' '.join([str(min_eligible_squares[key]) for key in SHAPEFILE_IDS]) \
      + ')') % min_eligible_squares['num_eligible_squares'])
max_eligible_squares = meta_info.sort_values('num_eligible_squares', ascending=False).iloc[0]
print(('Maximum eligible tiles: %.2f (' +  ' '.join([str(max_eligible_squares[key]) for key in SHAPEFILE_IDS]) \
      + ')') % max_eligible_squares['num_eligible_squares'])
print('Share of regions with capped tiles: %.2f' % \
      (len(meta_info[meta_info['count'] == MAX_TILES_PER_REGION])/len(meta_info)))
print('Share of tiles with full overlap: %.2f' % tiles['full_overlap'].mean())
print('Total number of tiles: %i' % len(tiles))

Minimum eligible tiles: 12.00 (New York 3803)
Maximum eligible tiles: 2236994.00 (Alaska 400)
Share of regions with capped tiles: 0.84
Share of tiles with full overlap: 0.75
Total number of tiles: 208802
